# Challenging Limitations: Using Deep Learning, Time Series Analysis, and Statistical Methods for Noise Reduction to Develop an Innovative Approach to Exoplanet Candidate Detection

In [1]:
#test
import ipyvolume as ipv
import numpy as np
x, y, z, u, v, w = np.random.random((6, 1000))*2-1
selected = np.random.randint(0, 1000, 100)
ipv.figure()
quiver = ipv.quiver(x, y, z, u, v, w, size=5, size_selected=8, selected=selected)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0, max=30, step=0.1)
size_selected = FloatSlider(min=0, max=30, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((quiver, 'size'), (size, 'value'))
jslink((quiver, 'size_selected'), (size_selected, 'value'))
jslink((quiver, 'color'), (color, 'value'))
jslink((quiver, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

VBox(children=(VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), geo='arrow', selected=array([202, 595, 181, 117, 177, 923, 578, 418, 753, 165, 519, 404, 140,
       290,  78, 942, 403, 545, 897, 371, 759, 598, 674,   1, 264, 595,
       924, 456,  39, 806, 150, 422, 926, 562, 690, 934, 416, 425, 956,
       840,  74, 950,  46, 918, 648, 741, 800, 254, 112, 835, 164, 322,
       758, 518, 355, 961,  19, 469,  52, 421, 850, 971, 941, 100, 484,
       580, 219, 992, 883, 368, 991, 386, 634, 521, 917, 528, 687, 306,
       481, 376, 439, 749, 538, 562,  17, 425, 763, 600, 623, 829, 874,
       592, 846, 988, 573, 231, 334,  87, 625, 121]), size_selected=array(8), vx=array([-0.96330438, -0.61292284, -0.10852618,  0.32411053, -0.59895719,
       -0.44569708,  0.57525864, -0.73560166,  0.65117488, -0.28753717,
       -0.21775058,  0.79443414,  0.83384923,  0.90659644,  0.24969517,
       -0.48576209,  0.33273148,  0.63310407,  0.05136643, -0.20264336,
        0.68420845,  0.36626946,  0.76641909, -0.69570327, -0.77883679,
        0.63048702, -0.62918383,  0.65829025,  0.83592624, -0.01107946,
        0.54236823,  0.56814558,  0.01221166,  0.38498796,  0.02522551,
       -0.83416233,  0.35862664,  0.58830441,  0.21427838, -0.99772451,
       -0.23167398,  0.71913265, -0.26513957, -0.48898304,  0.34756689,
       -0.22520263, -0.37487438,  0.57800249, -0.4198201 ,  0.05453902,
        0.68521839,  0.2232648 ,  0.8613885 , -0.42764462,  0.70565307,
       -0.51140886,  0.42975229, -0.56562776, -0.89721763,  0.07119812,
       -0.93484881, -0.23586136,  0.52535255, -0.14428368,  0.49329461,
        0.61591453,  0.53557648, -0.63240999, -0.50658207,  0.33754624,
       -0.73141066,  0.17612673, -0.89308021,  0.74621595, -0.10328767,
        0.17927289,  0.00495605,  0.88736143, -0.76270403, -0.63542647,
       -0.54082851, -0.19685079,  0.12501488,  0.19295957,  0.01723916,
        0.70889337,  0.12764732,  0.12561381,  0.15319581, -0.26290755,
        0.05629445,  0.36273928, -0.63926199,  0.86713252, -0.23893273,
       -0.36184803, -0.27167102, -0.77989875, -0.32577911, -0.20705504,
       -0.73848286,  0.05393   ,  0.15604607,  0.22346858, -0.91802329,
       -0.47408946,  0.55423318,  0.74809256,  0.09510572,  0.81947774,
       -0.29697009, -0.6131864 ,  0.95608273,  0.1746437 ,  0.66098837,
        0.56381133, -0.66912639, -0.50884491,  0.77280165,  0.51486565,
       -0.31520759, -0.52580003,  0.37183665,  0.6745934 , -0.33046192,
        0.65334933, -0.32979881,  0.7101855 ,  0.85304674,  0.92888037,
       -0.39271355, -0.83614106,  0.43042261,  0.85036297, -0.93816651,
        0.39878733, -0.90360481, -0.64349437, -0.40857319, -0.52076128,
        0.75173646,  0.08130242,  0.36928548,  0.19726942,  0.82712726,
        0.73308136,  0.61422533,  0.40527193, -0.7155065 , -0.74791082,
       -0.77697255, -0.57637311,  0.96662987, -0.21125428, -0.12211859,
        0.4624484 , -0.75304508,  0.68872877,  0.54770035, -0.73990502,
       -0.8698267 , -0.72109782,  0.72373819, -0.98136216, -0.66773162,
       -0.296578  ,  0.98477359,  0.30170449, -0.28685846, -0.24917426,
        0.46684606, -0.34825573,  0.44023109,  0.26107814,  0.14885367,
        0.7662719 , -0.96144191, -0.93826038, -0.73595393, -0.39252892,
        0.88892268,  0.10979595, -0.42835408, -0.58885396,  0.81077394,
        0.63641509, -0.06053506, -0.44499593, -0.38343334,  0.42238506,
       -0.55720948,  0.5589341 , -0.67148353,  0.76538795, -0.8558582 ,
       -0.82567518,  0.63178494, -0.41997125,  0.38629955, -0.0100118 ,
        0.20282636, -0.88156522, -0.890005  ,  0.8458837 , -0.1595654 ,
        0.20487108, -0.19786492, -0.34707502,  0.36708078,  0.02777251,
        0.29877929, -0.456

In [5]:
import pandas as pd
import numpy as np

#parse for dubious parallax data as indicated by HYG database
df = pd.read_csv('stardata_hyg_v2.csv')
df = df.loc[df['dist'] < 100000]    
df = df.reset_index(drop=True)
        
avg = 0
ra = df['ra']
dec = df['dec']
dist = df['dist']


def convert_to_cartesian(ra,dec,dist, n=500):
    x = np.empty(n)
    y = np.empty(n)
    z = np.empty(n)
    for i in range(0,n):
        z[i] = dist[i] * np.sin(dec[i])
        phi = dist[i] * np.cos(dec[i])
        y[i] = phi * np.sin(ra[i])
        x[i] = phi * np.cos(ra[i])
    return x,y,z
x,y,z = convert_to_cartesian(ra,dec,dist)


import ipyvolume as ipv
ipv.figure()
selected = (x[0], y[0], z[0])
scatter_a = ipv.scatter(x,y,z,size=0.4, size_selected = 0.6, marker="sphere", selected = selected)
from ipywidgets import FloatSlider, ColorPicker, VBox, jslink
size = FloatSlider(min=0.1, max=2, step=0.1)
opacity = FloatSlider(min = 0, max = 1, step=0.01)
color = ColorPicker()
color_selected = ColorPicker()
jslink((scatter_a, 'size'), (size, 'value'))
jslink((scatter_a, 'color'), (color, 'value'))
jslink((scatter_a, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, color, color_selected])

VBox(children=(VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), geo='sphere', selected=array([ 0.,  0.,  0.]), size=array(0.4), size_selected=array(0.6), x=array([  0.00000000e+00,   1.01838186e+02,   3.82022215e+01,
         1.76631094e+02,  -7.68267318e+00,  -2.49751373e+02,
        -3.81512433e+01,   2.16928718e+01,   1.46415063e+02,
         1.85644213e+02,   7.61300946e+01,  -2.35200016e+02,
        -5.14767561e+01,   1.92537707e+02,   4.52861631e+02,
        -1.08744751e+02,  -4.77183255e+01,  -2.74357676e+01,
         2.05120252e+02,  -3.16046290e+00,  -2.89112983e+01,
         1.94648543e+02,   6.33730283e+01,  -1.15011877e+01,
         7.75304141e+01,   6.82531013e+01,  -9.65896151e+01,
         1.61106523e+02,   1.65262702e+02,  -6.52204978e+01,
        -2.74018934e+02,  -3.40009437e+01,  -5.78238006e+01,
        -1.58897873e+01,  -5.02912253e+01,   1.76298359e+02,
        -3.81078564e+02,  -3.66631546e+01,  -5.08299524e+01,
        -2.28238829e+02,   1.23695217e+02,  -1.35074916e+02,
        -3.31014788e+02,  -6.22122831e+01,   7.35426603e+02,
         3.90822015e+01,  -6.63554612e+02,  -2.15265175e+02,
        -5.65944494e+01,  -1.07067740e+01,   3.19587685e+01,
        -4.92531054e+01,   2.47511693e+02,   2.52900219e+01,
         3.07494152e+01,  -1.08611545e+01,  -1.63689570e+02,
         4.01161343e+01,   5.55016343e+01,  -2.27664921e+02,
        -8.03081947e+00,  -1.26975251e+02,  -3.56715418e+00,
        -9.04819150e+00,   9.56650401e+01,   1.01783001e+02,
         8.30894546e+01,   8.48521465e+01,  -1.79624967e+02,
        -1.63346236e+01,   6.30486798e+02,   5.54860843e+01,
         4.53576029e+01,   3.93999439e+02,  -1.96557379e+02,
         5.38446545e+01,   9.55964128e+00,  -1.31428167e+02,
        -4.42204945e+01,   2.78502342e+02,   2.40577796e+02,
         3.06496256e+02,   2.08939665e+01,  -5.64616677e+02,
        -5.66503871e+01,   2.76332271e+02,  -1.13884370e+02,
         6.10107956e+01,   2.06736209e+01,   1.22962213e+02,
         5.67089529e+00,  -1.43415108e+02,  -1.49976988e+02,
        -6.90113014e+02,   1.15646514e+02,  -9.40792314e+01,
        -2.27314786e+02,   8.08412955e+01,   8.64329979e+01,
        -6.64802056e+01,   1.91563059e+02,   1.65918517e+02,
         1.50837229e+02,  -1.65835877e+01,  -3.70277155e+01,
        -1.04159321e+01,   9.15482778e+01,   2.57242852e+02,
        -9.03923493e+01,  -8.77870307e+02,  -7.76129437e+01,
         1.19596637e+02,   8.54374413e+01,  -2.23647379e+02,
        -6.42604138e+00,  -1.69110622e+02,  -3.60684208e+01,
        -1.48368356e+01,  -1.42640654e+02,   2.02305730e+01,
        -1.51333254e+01,   1.02543884e+02,   2.29136082e+02,
         8.59194622e+01,  -4.73809130e+02,  -1.60681240e+02,
         3.59881978e+02,   4.88725446e+01,   1.15013951e+02,
        -1.66006842e+01,  -1.12083176e+02,  -1.55824296e+01,
         5.95183300e+01,   5.09541221e+01,  -5.63704725e+01,
         2.70886022e+02,  -3.93028881e+01,  -1.38306419e+02,
         2.08506405e+02,   2.92899164e+01,  -2.52165654e+02,
         1.08322212e+02,  -1.87352567e+02,   1.76518706e+02,
        -9.72006626e+01,   1.27614878e+02,  -2.05021082e+02,
         2.58018360e+02,   8.48949298e+01,   1.50379002e+02,
        -5.80770815e+01,   4.61479370e+02,  -4.04269280e+02,
        -1.57427220e+02,   1.95345355e+02,  -2.79876261e+02,
        -7.20155115e+02,   1.06484769e+02,   1.99407715e+01,
        -6.63046233e+02,   1.02895493e+01,  -9.25402094e+01,
        -4.49663952e+00,  -1.18236312e+02,  -3.41595546e+02,
        -2.03761319e+02,   1.94614651e+02,   1.94725906e+02,
         1.96099822e+02,  -4.54884230e+01,   4.34848857e+01,
        -2.77434943e+02,  -1.92366914e+02,  -2.93629713e+02,
        -1.91524195e+01,   